<a href="https://colab.research.google.com/github/jakobengdahl/SCBDatahubDefinition/blob/main/draviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
#@title
from collections import OrderedDict
from typing import Dict

import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display


def create_plot(answers: Dict[str, int], answer_dict: Dict[int, str]) -> None:
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=[v for k, v in answers.items() if v != 4],
        theta=[k for k, v in answers.items() if v != 4],
        fill="toself",
        text=[k for k, v in answers.items() if v != 4],
        hoverinfo="text"
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                tickmode="array",
                tickvals=[0, 1, 2, 3],
                ticktext=list(answer_dict.values()),
                range=[0, 3],
                categoryorder="category ascending"
            ),
            angularaxis=dict(
                direction="clockwise"
            )
        ),
        showlegend=False,
        font=dict(size=12)
    )

    fig.show()


def print_questions(questions: pd.DataFrame, answer_dict: Dict[int, str]) -> None:
    answers = OrderedDict()
    for i in range(len(questions)):
        qstring = "%i. %s" % (questions["question_id"][i], questions["question_text"][i])
        qshort = "%i. %s" % (questions["question_id"][i], questions["shorthand"][i])
        ans_val = widgets.RadioButtons(
            options=list(answer_dict.values()),
            description=qstring,
            disabled=False
        )
        answers[qshort] = ans_val
        display(ans_val)

    submit_button = widgets.Button(
        description='Submit',
        disabled=False,
        button_style='',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to submit answers',
        icon='check'
    )

    def on_submit(b):
        answer_values = {k: v.value for k, v in answers.items()}
        create_plot(answer_values, answer_dict)

    submit_button.on_click(on_submit)
    display(submit_button)


def main() -> None:
    langs: Dict[str, str] = {
        'English': 'en',
        'Svenska': 'sv'
    }

    language = widgets.Dropdown(
        options=list(langs.keys()),
        description='Language:',
        value='English',
    )
    display(language)

    dd = OrderedDict()
    if language.value == "English":
        title = "Data Readiness Assessment"
        dd: Dict[int, str] = {
            0: "Don't know",
            1: "No",
            2: "Partially",
            3: "Yes",
            4: "Not relevant"
        }

        q_options: Dict[int, str] = {
            0: "None",
            1: "Default questions",
            2: "User-specified questions"
        }
        q_prompt = "Select source of questions"
        upload_prompt = "Upload csv file with questions"

    elif language.value == "Svenska":
        title = "Bedömning av databeredskap"
        dd: Dict[int, str] = {
            0: "Vet inte",
            1: "Nej",
            2: "Delvis",
            3: "Ja",
            4: "Inte relevant"
        }

        q_options: Dict[int, str] = {
            0: "None",
            1: "Standardfrågor",
            2: "Egna frågor"
        }
        q_prompt = "Välj källa till frågor"
        upload_prompt = "Ladda upp csv-fil med frågor"
    else:
        raise RuntimeError(f"Unknown language: '{language.value}'")

    q_mode = widgets.Dropdown(
        options=list(q_options.values()),
        description=q_prompt,
        value="Default questions"
    )
    display(q_mode)

    mode = [k for k, v in q_options.items() if v == q_mode.value][0]

    if mode == 0:
        input_file = None
    elif mode == 1:
        input_file = "data/defaultq_%s.csv" % langs[language.value]
    elif mode == 2:
        input_file = widgets.FileUpload(
            description=upload_prompt,
            accept='.csv',
            multiple=False
        )
        display(input_file)

    if input_file is not None and mode != 0:
        if isinstance(input_file, str):
            qq = pd.read_csv(input_file)
        else:
            # extract file from file upload widget
            uploaded_file = next(iter(input_file.values()))
            qq = pd.read_csv(uploaded_file)

        try:
            print_questions(qq, dd)
        except KeyError:
            print("Not a valid question file. Expected columns question_id, shorthand, question_text. Instead got %s." % ", ".join(
                qq.columns))


if __name__ == "__main__":
    main()


Dropdown(description='Language:', options=('English', 'Svenska'), value='English')

Dropdown(description='Select source of questions', index=1, options=('None', 'Default questions', 'User-specif…

RadioButtons(description='1. Do you have programmatic access to the data?', options=("Don't know", 'No', 'Part…

RadioButtons(description='2. Are your licenses in order?', options=("Don't know", 'No', 'Partially', 'Yes', 'N…

RadioButtons(description='3. Do you have lawful access to the data?', options=("Don't know", 'No', 'Partially'…

...

RadioButtons(description='12. Is your organization prepared to handle more data like this beyond the scope of …

RadioButtons(description='13. Is the data secured?', options=("Don't know", 'No', 'Partially', 'Yes', 'Not rel…

RadioButtons(description='14. Is it safe to share the data with others?', options=("Don't know", 'No', 'Partia…

RadioButtons(description='15. Are you allowed to share the data with others?', options=("Don't know", 'No', 'P…

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to submit answers')